# О том, как учат сложные сетки 

In [4]:
import numpy as np
import tensorflow as tf
tf.__version__ 

'2.0.0'

In [5]:
from tensorflow.keras import Sequential, Model
keras, L = tf.keras, tf.keras.layers

В этой тетрадке немного поработаем с градусами по цельсию и фаренгейту! Снова попробуем восстановить формулу 

$$ f = c \times 1.8 + 32 $$

In [6]:
celsius    = np.array([-40, -10,  0,  8, 15, 22,  38],  dtype=float)
fahrenheit = np.array([-40,  14, 32, 46, 59, 72, 100],  dtype=float)

for i,c in enumerate(celsius):
    print("{} degrees Celsius = {} degrees Fahrenheit".format(c, fahrenheit[i]))

-40.0 degrees Celsius = -40.0 degrees Fahrenheit
-10.0 degrees Celsius = 14.0 degrees Fahrenheit
0.0 degrees Celsius = 32.0 degrees Fahrenheit
8.0 degrees Celsius = 46.0 degrees Fahrenheit
15.0 degrees Celsius = 59.0 degrees Fahrenheit
22.0 degrees Celsius = 72.0 degrees Fahrenheit
38.0 degrees Celsius = 100.0 degrees Fahrenheit


In [7]:
# транспонировали выборку
x_train = celsius[:,None]
y_train = fahrenheit[:,None]

# 1. Как мы обучались до этого 

На Keras было всё совсем просто. 

In [8]:
model = Sequential()
model.add(L.Dense(1))

opt = tf.keras.optimizers.Adam( )

model.compile(loss='mse', optimizer=opt)
model.fit(x_train, y_train, validation_split=0.2, epochs=2, verbose=1)

Train on 5 samples, validate on 2 samples
Epoch 1/2
5/5 [==============================] - 1s 109ms/sample - loss: 3296.3652 - val_loss: 14434.8848
Epoch 2/2
5/5 [==============================] - 0s 2ms/sample - loss: 3294.3977 - val_loss: 14427.2090


После мы сказали, что иногда хочется собирать более сложные модели, а ещё жёстко контролировать обучение, и посмотрели на Tensorflow.

In [9]:
a = tf.Variable(tf.random.normal([1]))
b = tf.Variable(tf.random.normal([1]))

# Наша модель
def linear_regression(x):
    return a + b*x

# Ошибка для модели
def mean_square(y_pred, y_true):
    return tf.reduce_mean((y_pred-y_true)**2)

# оптимизатор 
optimizer = tf.optimizers.SGD(learning_rate=0.001)

# процесс оптимизации
def model_train(X, Y):

    # находим loss и пробрасываем градиент
    with tf.GradientTape() as g:
        pred = linear_regression(X)
        loss = mean_square(pred, Y)

    # Вычисляем градиенты
    gradients = g.gradient(loss, [a, b])
    
    # Обновляем веса a и b в ходе одной итерации спуска 
    optimizer.apply_gradients(zip(gradients, [a, b]))
    pass

In [10]:
#Обучение
epochs = 100 # число эпох 

for i in range(epochs):
    
    # Делаем щаг градиентного спуска 
    model_train(celsius, fahrenheit)
    
    # Каждую сотую итерацию следим за тем, что произошло
    if i%10 == 0:
        y_pred = linear_regression(celsius)
        loss_val = mean_square(y_pred, fahrenheit)
        print("step: %i, loss: %f, a: %f, b: %f" % (i, loss_val, a.numpy(), b.numpy()))

step: 0, loss: 978.607117, a: 0.097504, b: 2.151965
step: 10, loss: 937.793457, a: 0.703292, b: 2.061671
step: 20, loss: 902.426270, a: 1.298252, b: 2.056649
step: 30, loss: 868.392883, a: 1.881884, b: 2.051724
step: 40, loss: 835.642761, a: 2.454405, b: 2.046893
step: 50, loss: 804.128357, a: 3.016025, b: 2.042153
step: 60, loss: 773.802246, a: 3.566952, b: 2.037503
step: 70, loss: 744.620117, a: 4.107390, b: 2.032943
step: 80, loss: 716.538513, a: 4.637538, b: 2.028468
step: 90, loss: 689.516052, a: 5.157593, b: 2.024080


# 2. Как обычно обучаются на Tensorflow 

Для того, чтобы было удобнее, модели заворачивают в полноценные классы. 

In [11]:
class Super_puper_neural_net(Model):
    
    def __init__(self, n_hidden_neurons):
        super(Super_puper_neural_net, self).__init__()
        self.fc1 = L.Dense(n_hidden_neurons, kernel_initializer='glorot_uniform',
                           activation='relu', trainable=True)
        self.fc2 = L.Dense(n_hidden_neurons, kernel_initializer='glorot_uniform',
                           trainable=True)

    def encode(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [12]:
model = Super_puper_neural_net(1)
model.encode(x_train)

W1010 17:28:37.258396 140736665199552 base_layer.py:1814] Layer dense_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Tensor: id=4954, shape=(7, 1), dtype=float32, numpy=
array([[0.       ],
       [0.       ],
       [0.       ],
       [0.5753724],
       [1.0788232],
       [1.5822741],
       [2.7330189]], dtype=float32)>

In [17]:
model1 = Super_puper_neural_net(1)
model2 = Super_puper_neural_net(1)

x1 = model1.encode(x_train)
x2 = model2.encode(x_train)

W1010 17:30:02.896837 140736665199552 base_layer.py:1814] Layer dense_19 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

W1010 17:30:02.904983 140736665199552 base_layer.py:1814] Layer dense_21 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults t

In [18]:
# Список из переменных
model.variables

[<tf.Variable 'dense_1/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.43147695]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>,
 <tf.Variable 'dense_2/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.16668689]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

Можно легко подменять одни переменные какими-нибудь своими. Это довольно удобно, когда хочется позаимствовать чужие веса. В будущем мы будем этим активно заниматься. 

In [19]:
# Ошибка для модели
def mean_square(y_pred, y_true):
    return tf.reduce_mean((y_pred-y_true)**2)

# оптимизатор 
optimizer = tf.optimizers.SGD(learning_rate=0.001)

# процесс оптимизации
def model_train(X, Y):

    # находим loss и пробрасываем градиент
    with tf.GradientTape() as g:
        pred = model.encode(X)
        loss = mean_square(pred, Y)

    # Вычисляем градиенты
    gradients = g.gradient(loss, model.variables)
    
    # Обновляем веса a и b в ходе одной итерации спуска 
    optimizer.apply_gradients(zip(gradients, model.variables))
    pass

In [91]:
#Обучение
epochs = 100 # число эпох 

for i in range(epochs):
    
    # Делаем щаг градиентного спуска 
    model_train(x_train, y_train)
    
    # Каждую сотую итерацию следим за тем, что произошло
    if i%10 == 0:
        y_pred = model.encode(x_train)
        loss_val = mean_square(y_pred, y_train)
        print("step: %i, loss: %f" % (i, loss_val))

step: 0, loss: 3272.454346
step: 10, loss: 3100.122559
step: 20, loss: 3017.487549
step: 30, loss: 2937.423096
step: 40, loss: 2859.850830
step: 50, loss: 2784.693604
step: 60, loss: 2711.876953
step: 70, loss: 2641.329346
step: 80, loss: 2572.979980
step: 90, loss: 2506.761230


# 3. Свой слой на Tensorflow для Keras

Новые слои можно писать на основе керасовского класса `Layer`. Если прописать `help(tf.keras.layers.Layer)`, можно почитать про него. Если в кратце, нужно реализовать три части: 

* Конструктор, в нём мы описываем гиперпараметры 
* Метод `build`, в которм мы описываем все переменные 
* Метод `call`, который делает forward pass

In [20]:
class MyLinear(L.Layer):
    
    # Задаём консруктор 
    def __init__(self, units=32):
        super(MyLinear, self).__init__()  # чтобы коректно унаследовались методы
        self.units = units                # число нейронов

    def build(self, input_shape):
        # add_weight внутри build то же самое что и Variable, но совместимо с Keras
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal', 
                                 trainable=True)
        
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal', 
                                 trainable=True)

    # Применение 
    def call(self, inputs):
        # сразу делаем и линейное преобразование и ReLU (а почему бы и нет)
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b) 

In [21]:
model = Sequential()
model.add(MyLinear())  # добавили свой слой! 
model.add(L.Dense(1))

In [22]:
opt = tf.keras.optimizers.Adam( )
model.compile(loss='mse', optimizer=opt)

In [23]:
model.fit(x_train, y_train, validation_split=0.2, epochs=2, verbose=1)

Train on 5 samples, validate on 2 samples
Epoch 1/2
5/5 [==============================] - 0s 65ms/sample - loss: 1639.6702 - val_loss: 7551.9795
Epoch 2/2
5/5 [==============================] - 0s 2ms/sample - loss: 1636.3309 - val_loss: 7542.4536


По аналогии с помощью класса `Model` можно делать полноценные слои-модели. Этим мы тоже будем позже заниматься. Почитать про свои слои и модели можно подробнее [вот тут, в документации.](https://www.tensorflow.org/beta/guide/keras/custom_layers_and_models)